# 유사문제 모델

In [318]:
import pandas as pd
tmp_df_orig = pd.read_pickle("<전체문제>")
tmp_df_24acm = pd.read_pickle("<24임종평>")

tmp_df_all = pd.concat([tmp_df_orig,tmp_df_24acm])


In [319]:
from transformers import AutoTokenizer
from rank_bm25 import BM25Okapi

class MultilingualBM25Retriever:
    def __init__(self, documents, document_ids, model_name='bert-base-multilingual-cased'):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.documents = documents
        self.document_ids = document_ids
        self.tokenized_documents = [self.tokenize(doc) for doc in documents]
        self.bm25 = BM25Okapi(self.tokenized_documents)
    
    def tokenize(self, text):
        tokens = self.tokenizer.tokenize(text)
        return tokens
    
    def search(self, query, current_id=None, top_k=2):
        tokenized_query = self.tokenize(query)
        print("Tokenized query:", tokenized_query)
        scores = self.bm25.get_scores(tokenized_query)

        # 자기 자신을 제외하고 상위 top_k 추출
        filtered_results = [
            (i, scores[i]) for i in range(len(scores)) if self.document_ids[i] != current_id
        ]
        top_n = sorted(filtered_results, key=lambda x: x[1], reverse=True)[:top_k]
        return [(self.document_ids[i], self.documents[i], score) for i, score in top_n]

# 사용 예시
def find_similar_questions(wrong_numbers, tmp_df_all):
    documents = tmp_df_all['문제'].tolist()
    document_ids = tmp_df_all['문제고유번호'].tolist()

    retriever = MultilingualBM25Retriever(documents, document_ids)

    results_by_wrong_number = {}

    for wrong_number in wrong_numbers:
        # 틀린 문제 텍스트를 검색 쿼리로 사용
        if wrong_number not in document_ids:
            print(f"문제고유번호 {wrong_number}는 데이터에서 찾을 수 없습니다.")
            results_by_wrong_number[wrong_number] = []
            continue
        
        query = tmp_df_all[tmp_df_all['문제고유번호'] == wrong_number]['문제'].iloc[0]
        print(f"\n문제고유번호 {wrong_number} 쿼리: {query}")
        results = retriever.search(query, current_id=wrong_number, top_k=2)
        results_by_wrong_number[wrong_number] = [
            {"문제고유번호": doc_id, "문제": doc, "유사도": score}
            for doc_id, doc, score in results
        ]

    return results_by_wrong_number

# 틀린 문제와 유사한 문제 찾기 실행
wrong_numbers = ['24000101', '23000255']  # 틀린 문제 고유번호 리스트
results = find_similar_questions(wrong_numbers, tmp_df_all)

# 결과 출력
for wrong_number, similar_questions in results.items():
    print(f"\n문제고유번호 {wrong_number}와 가장 유사한 2문제:")
    for idx, question in enumerate(similar_questions):
        print(f"{idx + 1}문제고유번호: {question['문제고유번호']}")
        print(f"유사도: {question['유사도']:.4f}")
        print(f"문제: {question['문제']}")
results


문제고유번호 24000101 쿼리: 인체면역결핍바이러스에 감염된 '갑'이 자택에서 사망하였다. ‘갑'의 보호자가 ‘갑'에 대한 시체검안서를 의사에게 요청하였다. 「후전성면역결핍증 예방법」상 '갑'의 시체를 검안한 의사는 검안 사실을 누구에게 신고하여야 하는가?
Tokenized query: ['인', '##체', '##면', '##역', '##결', '##핍', '##바', '##이', '##러', '##스', '##에', '감', '##염', '##된', "'", '갑', "'", '이', '자', '##택', '##에서', '사', '##망', '##하였다', '.', '[UNK]', '갑', "'", '의', '보', '##호', '##자가', '[UNK]', '갑', "'", '에', '대한', '시', '##체', '##검', '##안', '##서를', '의', '##사에', '##게', '요', '##청', '##하였다', '.', '「', '후', '##전', '##성', '##면', '##역', '##결', '##핍', '##증', '예', '##방', '##법', '」', '상', "'", '갑', "'", '의', '시', '##체를', '검', '##안', '##한', '의', '##사는', '검', '##안', '사', '##실을', '누', '##구', '##에게', '신', '##고', '##하여', '##야', '하는', '##가', '?']

문제고유번호 23000255 쿼리: 63세 여자가 3년 전부터 양쪽 손목과 손가락이 아프다며 병원에 왔다. 손가락 통증과 뻣뻣함은 아침에 심하고 오후에 호전되었으며, 1년 전부터 손에 변형이 생겼다. 혈액 검사 결과는 다음과 같다. 치료는?
Tokenized query: ['63', '##세', '여자', '##가', '3', '##년', '전', '##부터', '양', '##쪽', '손', '##목', '##과', '손', '##가', '##락', '##이', '아', '##프', '##다

{'24000101': [{'문제고유번호': '23000111',
   '문제': '후천성면역결핍증에 관한 검진을 시행하는 검사기관에서 인체면역결핍바이러스에 감염된 자를 확인하였다. 「후천성면역 결핍증 예방법」에 따라 감염인을 진단한 의사는 이를 누구에게 신고해야 하는가?',
   '유사도': 170.88162753616896},
  {'문제고유번호': '24000113',
   '문제': "'A' 병원에서 근무하는 의사 '갑'은 정보통신기술을 활용하여 먼 곳에 소재한 'B' 병원 소속 의사 ’을'에게 원격으로 의료기술을 지원하였다. 원격의료에 따른 의료행위는 의사 ’을'이 수행하였다. 해당 진료 중 의료사고가 발생하였다. 'B' 병원 원장은 '병'이다. 「의료법」상 의사 ‘갑'의 과실을 인정할 만한 명백한 근거가 없을 때 해당 진료에 대한 책임은 누구에게 있는 것으로 보는가?",
   '유사도': 152.0152462085728}],
 '23000255': [{'문제고유번호': '20000331',
   '문제': '63세 여자가 2년 전부터 양쪽 손가락과 손목에 통증이 있다며 병원에 왔다. 아침경직은 2시간 동안 지속되고, 양쪽 손목에 압통이 있다. 손 X선 사진이다. 혈액검사 결과는 다음과 같다. 치료는?',
   '유사도': 120.30849478410299},
  {'문제고유번호': '19000547',
   '문제': '61세 여자가 2년 전부터 양쪽 손가락과 손목이 아프다며 병원에 왔다. 통증은 아침에 심하고, 활동하면 오후부터 서서히 호전되었다. 손 사진과 손 X선 사진이다. 혈액검사 결과는 다음과 같다. 적합한 진단을 고르시오 (한 가지).',
   '유사도': 117.0995029612565}]}

# 오답노트 파일생성

In [320]:

from datetime import datetime
time = datetime.now()
print(time)
import pandas as pd
import random



2025-01-01 20:29:34.149762


In [322]:
# 기본 Python 라이브러리
import io
import re
import os
import json
import time
import math

# 데이터 처리 및 분석
import pandas as pd
from tqdm import tqdm

# 날짜 및 시간 처리
from datetime import datetime

# 이미지 처리
from PIL import Image

# 문서 처리
from docx import Document
from docx.shared import Inches, Pt, RGBColor
from docx.enum.style import WD_STYLE_TYPE
from docx.enum.text import WD_BREAK
import fitz  # PyMuPDF

# 네트워크 요청
import requests

# Windows COM 인터페이스
import win32com.client as win32

# GUI
import tkinter as tk
from tkinter import ttk
from tkinter import simpledialog

In [323]:
def get_exam_type(row):
	if row['시험종류'] == '00':
		return f"{row['시험연도']}국시"
	else:
		return f"임종평{row['시험연도']}-{int(row['시험종류'])}"

def format_answers(ans_list):
    def circled_number(text):
        parts = text.split()
        if parts[0].isdigit():
            circled_num = chr(0x245F + int(parts[0]))
            return f"{circled_num} {' '.join(parts[1:])}"
        return text
    
    return [circled_number(a) for a in ans_list]

In [324]:
def insert_image_into_word(doc, img_url):
    """
    워드 문서에 직접 이미지를 삽입.

    Parameters:
    - doc: 워드 문서 객체
    - img_url: 이미지의 URL
    """
    try:
        response = requests.get(img_url)
        if response.status_code == 200:
            # 이미지 데이터 읽기
            image_stream = io.BytesIO(response.content)
            image = Image.open(image_stream)

            # 이미지가 RGBA 혹은 LA 모드일 경우 RGB로 변환
            if image.mode in ['RGBA', 'LA']:
                background = Image.new(image.mode[:-1], image.size, (255, 255, 255))
                background.paste(image, image.split()[-1])
                image = background.convert('RGB')
            
            # 이미지 크기를 조정하기 위한 최대치 설정
            max_width_inches = 3.8
            max_height_inches = 1.9

            # 이미지 원본의 가로, 세로 크기를 인치로 계산 (기본 DPI를 96으로 가정)
            dpi = image.info.get('dpi', (96, 96))
            original_width_inches = image.width / dpi[0]
            original_height_inches = image.height / dpi[1]

            # 가로세로비에서 가로가 세로보다 1.3배 이상 클 경우 가로를 최대로 설정
            aspect_ratio = original_width_inches / original_height_inches
            if aspect_ratio >= 1.3:
                final_width = max_width_inches
                scale_ratio = final_width / original_width_inches
                final_height = original_height_inches * scale_ratio
            else:
                # 가로, 세로 비율에 따라 적절한 축소 비율 계산
                width_ratio = max_width_inches / original_width_inches
                height_ratio = max_height_inches / original_height_inches
                scale_ratio = min(width_ratio, height_ratio)
                final_width = original_width_inches * scale_ratio
                final_height = original_height_inches * scale_ratio
            
            # 임시 파일로 저장 후 다시 문서에 삽입
            with io.BytesIO() as img_byte_stream:
                image.save(img_byte_stream, 'JPEG')
                img_byte_stream.seek(0)
                doc.add_picture(img_byte_stream, width=Inches(final_width), height=Inches(final_height))

    except Exception as e:
        print(f"워드 문서에 이미지 삽입 중 오류 발생: {e}, URL: {img_url}")


In [325]:
## 단 설정
WNS_COLS_NUM = "{http://schemas.openxmlformats.org/wordprocessingml/2006/main}num"
def set_number_of_columns(section, cols):
    """ sets number of columns through xpath. """
    section._sectPr.xpath("./w:cols")[0].set(WNS_COLS_NUM, str(cols))

In [326]:
def create_title_style(doc):
    title_style = doc.styles.add_style('TitleStyle', WD_STYLE_TYPE.PARAGRAPH)
    title_font = title_style.font
    title_font.name = 'Malgun Gothic'
    title_font.size = Pt(11)
    title_font.bold = True
    title_font.color.rgb = RGBColor(0, 0, 255)  # 파란색
    return title_style

def create_question_style(doc):
    question_style = doc.styles.add_style('QuestionStyle', WD_STYLE_TYPE.PARAGRAPH)
    question_font = question_style.font
    question_font.name = 'Malgun Gothic'
    question_font.size = Pt(9)
    question_font.color.rgb = RGBColor(0, 0, 0)  # 검은색
    question_style.paragraph_format.line_spacing = 1
    question_style.paragraph_format.space_before = Pt(0)
    question_style.paragraph_format.space_after = Pt(0)
    return question_style

def create_table_data_style(doc):
    table_data_style = doc.styles.add_style('TableDataStyle', WD_STYLE_TYPE.PARAGRAPH)
    table_data_font = table_data_style.font
    table_data_font.name = 'Malgun Gothic'
    table_data_font.size = Pt(8)
    table_data_font.color.rgb = RGBColor(0, 0, 0)  # 검은색
    table_data_style.paragraph_format.line_spacing = 1
    table_data_style.paragraph_format.space_before = Pt(0)
    table_data_style.paragraph_format.space_after = Pt(0)
    return table_data_style

def create_list_number_style(doc):
    list_number_style = doc.styles.add_style('ListNumberStyle', WD_STYLE_TYPE.PARAGRAPH)
    original_style = doc.styles['List Number']    
    font = list_number_style.font
    font.name = 'Malgun Gothic'  # 폰트 이름 설정
    font.size = Pt(10)  # 폰트 크기 설정
    font.bold = True  # 굵게 설정
    font.italic = False  # 이탤릭 해제
    font.color.rgb = RGBColor(0, 0, 0)  # 폰트 색상 설정 (검은색)    
    list_number_style.base_style = original_style

    return list_number_style

In [327]:
def set_style(doc):
	title_style = create_title_style(doc)
	question_style = create_question_style(doc)
	answer_data_style = question_style
	lab_data_style = question_style
	table_data_style = create_table_data_style(doc)
	option_data_style = question_style
	list_numb_style = create_list_number_style(doc)
	
	final_styles = [
		title_style,
		question_style,
		answer_data_style,
		lab_data_style,
		table_data_style,
		option_data_style,
		list_numb_style
		]
	return final_styles

In [328]:
def getColorClass(정답률):
    if math.isnan(정답률):
        return '정답률-nan'
    if 정답률 >= 80:
        return '정답률-high'
    if 정답률 >= 60:
        return '정답률-medium'
    return '정답률-low'

In [329]:
from docx.oxml import OxmlElement
from docx.oxml.ns import qn
from docx.shared import Pt

def add_hyperlink(paragraph, url, text, styles=None):
    """
    워드 문서 단락에 하이퍼링크를 추가하는 함수.
    
    Args:
        paragraph: python-docx Paragraph 객체
        url: 하이퍼링크 URL
        text: 표시될 텍스트
        styles: 텍스트 스타일 (선택 사항)
    """
    # Create the w:hyperlink tag and add required attributes
    part = paragraph.part
    r_id = part.relate_to(url, "http://schemas.openxmlformats.org/officeDocument/2006/relationships/hyperlink", is_external=True)
    hyperlink = OxmlElement("w:hyperlink")
    hyperlink.set(qn("r:id"), r_id)

    # Create a w:r element (run) and add it to the hyperlink
    run = OxmlElement("w:r")
    r_pr = OxmlElement("w:rPr")  # Add run properties
    run.append(r_pr)

    # Add text to the run
    t = OxmlElement("w:t")
    t.text = text
    run.append(t)
    hyperlink.append(run)

    # Add the hyperlink to the paragraph
    paragraph._p.append(hyperlink)

    # Apply styles if provided
    if styles:
        run = paragraph.add_run()
        run.font.size = Pt(styles.get('size', 8))
        run.font.color.rgb = styles.get('color', None)


In [330]:
def insert_question_to_doc(doc, df, 문제고유번호, styles, is_원래문제=True):

    row = df[df['문제고유번호'] == 문제고유번호].iloc[0]
    question_info = {
        "impression": '',
        "differential_diagnosis": '',
        "first_sentence": f"{row['문제'].split('. ')[0]}",
        "question_info": f'{row["exam_type"]} {int(row["교시"])}교시 #{row["문제번호"]}',
        "allen_info": f'{row["대단원"]}-{row["소단원"]} #{row["알렌문제번호"]}',
        "answer": row['정답들'],
        "특이고유번호": row['문제고유번호'],
        "allen_link": f'https://<URL 링크>/study/chapter/{row["소단원번호"]}/{row["알렌문제번호"]}'
    }

    if row['imp']:
        impression = f"{', '.join(row['imp'])}"
        question_info['impression'] = impression
    if row['ddx']:
        differential_diagnosis = f"{','.join(row['ddx'])}"
        question_info['differential_diagnosis'] = differential_diagnosis
    if is_원래문제:
        doc.add_paragraph(f'{question_info["question_info"]}', style=styles['number_list_sty'])
    else:
        doc.add_paragraph(f'{question_info["question_info"]}', style=styles['q_sty'])


    # Add Allen link as a hyperlink
    paragraph = doc.add_paragraph()
    add_hyperlink(paragraph, question_info['allen_link'], "링크", styles={'size': 8})

    question_text = f"{row['문제']}"
    doc.add_paragraph(question_text, style=styles['q_sty'])

    lab_data = '\n'.join(row['lab_data'])
    if lab_data:
        lab_table = doc.add_table(rows=1, cols=1)
        lab_table.style = 'Table Grid'
        cell = lab_table.cell(0, 0)
        lab_paragraph = cell.paragraphs[0]
        lab_paragraph.style = styles['lab_sty']
        lab_paragraph.add_run(lab_data)

    table_data = row['table']
    if table_data:
        print(table_data)
        try:
            # `table_data`가 리스트인지 확인
            if isinstance(table_data, list):
                for data_frame in table_data:
                    # 데이터프레임 확인
                    if isinstance(data_frame, pd.DataFrame):
                        table = doc.add_table(rows=len(data_frame), cols=len(data_frame.columns))
                        table.style = 'Table Grid'
                        for row_index, data_row in data_frame.iterrows():
                            for col_index, value in enumerate(data_row):
                                cell_value = '' if pd.isna(value) else str(value)
                                cell = table.cell(row_index, col_index)
                                table_paragraph = cell.paragraphs[0]
                                table_paragraph.style = styles['table_sty']
                                table_paragraph.add_run(cell_value)
                    else:
                        # `data_frame`이 데이터프레임이 아니면 오류 메시지 출력
                        print(f"Warning: Expected DataFrame, got {type(data_frame)}: {data_frame}")
            else:
                # `table_data`가 리스트가 아니면 오류 메시지 출력
                print(f"Warning: Expected list of DataFrames, got {type(table_data)}")
        
        except Exception as e:
            # 오류 발생 시 `table_data`를 문자열로 처리
            print(f"Error processing table_data: {e}")
            if isinstance(table_data, str):
                table_text = table_data  # 이미 문자열이면 그대로 사용
            else:
                table_text = "\n".join(str(item) for item in table_data)  # 리스트나 다른 타입 처리

            # 워드 문서에 문자열로 추가
            table = doc.add_table(rows=1, cols=1)
            table.style = 'Table Grid'
            cell = table.cell(0, 0)
            cell.text = table_text


    for url in row['사진들']:
        insert_image_into_word(doc, url)

    choices = row['선지들']
    formatted_choices = [f"{chr(0x245F + i + 1)} {option}" for i, option in enumerate(choices)]
    option_text = '\n'.join(formatted_choices)
    doc.add_paragraph(option_text, style=styles['opt_sty'])
    doc.add_paragraph(style=styles['q_sty'])

    return question_info

In [331]:

def making_word_file(df, 고유번호리스트, save_path, current_datetime, 시험지제목, 틀린문제모델결과):
    doc = Document()

    # 첫 번째 섹션 설정
    section1 = doc.sections[0]
    section1.top_margin = Inches(0.3)
    section1.bottom_margin = Inches(0.3)
    section1.left_margin = Inches(0.3)
    section1.right_margin = Inches(0.3)
    set_number_of_columns(section1, 2)

    style_list = set_style(doc)

    styles = {
        't_sty': style_list[0],
        'q_sty': style_list[1],
        'a_sty': style_list[2],
        'lab_sty': style_list[3],
        'table_sty': style_list[4],
        'opt_sty': style_list[5],
        'number_list_sty': style_list[6]
    }

    doc.add_paragraph(f"{시험지제목}", style=styles['t_sty'])
    doc.add_paragraph(f"시험날짜 | {datetime.now().strftime('%Y-%m-%d')}", style=styles['t_sty'])
    doc.add_paragraph(f"총 문항 수: {len(고유번호리스트)}", style=styles['t_sty'])
    doc.add_page_break()

    question_data = {}
    for 문제고유번호 in 고유번호리스트:
        # 원래 문제 추가
        question_data[문제고유번호] = insert_question_to_doc(doc, df, 문제고유번호, styles)
        

        if 틀린문제모델결과:
            paragraph = doc.add_paragraph()  # 새로운 단락 추가
            run = paragraph.add_run()       # Run 객체 생성
            run.add_break(WD_BREAK.COLUMN)  # 단 나누                    
            # 비슷한 문제를 다른 단에 추가
            for sim_qo in 틀린문제모델결과[문제고유번호]:
                insert_question_to_doc(doc, df, sim_qo['문제고유번호'], styles, is_원래문제=False)  # 비슷한 문제 추가

        # 페이지 나누기
        doc.add_page_break()

    doc.save(save_path)

    print(fr"저장경로: {save_path}")
    return question_data

In [332]:
def convert_doc_to_pdf(doc_path, pdf_path, max_retries=3):
    attempt = 0
    while attempt < max_retries:
        try:
            # Word 애플리케이션을 시작
            word = win32.gencache.EnsureDispatch('Word.Application')
            doc = word.Documents.Open(doc_path)
            doc.Activate()

            # Word 문서를 PDF로 저장
            word.ActiveDocument.ExportAsFixedFormat(pdf_path, ExportFormat=17)  # 17은 PDF를 의미
            doc.Close(False)

            # Word 애플리케이션 종료
            word.Quit()
            print(f"PDF 파일 저장 성공: {pdf_path}")
            return  # 성공 시 함수 종료

        except Exception as e:
            attempt += 1
            if attempt == max_retries:
                print("최대 재시도 횟수에 도달했습니다. 실패했습니다.")
            time.sleep(1)  # 잠시 대기 후 재시도


In [333]:
def get_correct_rate(file, 문제번호들):
    df = pd.read_csv(file, sep='\t')
    df['문제고유번호'] = df['문제고유번호'].astype(str)  # 여기에 추가
    
    dic = {}
    for idx, qno in enumerate(문제번호들):
        matching_row = df[df['문제고유번호'] == str(qno)]  # str(qno)로 변경
    
    return dic

In [334]:
def save_to_txt(number_list, filename):
    with open(filename, "w") as file:
        for numbers in number_list:
            for i in range(0, len(numbers), 10):
                line = ",".join(map(str, numbers[i:i+10]))
                file.write(line + ",\n")
            file.write("\n")

In [335]:
def sort_allen_except_num(df):
    adf = df[df['시험종류'].isin(['00',0])]
    bdf = df[df['시험종류'].isin(['01','02',1,2])]
    adf = adf.sort_values(by=['시험연도', '문제고유번호'], ascending=[False, True])
    bdf = bdf.sort_values(by=['시험연도', '시험종류','문제고유번호'], ascending=[False, False,True])
    return pd.concat([adf, bdf], ignore_index=True)


def main(고유번호필터링_1, 고유번호필터링_2, 파일명앞에붙을이름, 시험지제목, save_folder, file):
    tmp_df_orig = pd.read_pickle("<전체문제>")

    df = pd.read_pickle(file)
    df = pd.concat([tmp_df_orig, df])

    df['exam_type'] = df.apply(get_exam_type, axis=1)
    df['문제고유번호'] = df['문제고유번호'].astype(str)

    # 첫 번째 입력 처리
    고유번호필터링_1 = [str(x) for x in 고유번호필터링_1]
    df1 = df[df['문제고유번호'].isin(고유번호필터링_1)]
    df1 = df1.set_index('문제고유번호').loc[고유번호필터링_1].reset_index()
    df1 = sort_allen_except_num(df1)
    df1 = df1.reset_index()
    고유번호정렬_1 = df1['문제고유번호'].tolist()

    # 두 번째 입력 처리
    고유번호필터링_2 = [str(x) for x in 고유번호필터링_2]
    df2 = df[df['문제고유번호'].isin(고유번호필터링_2)]
    df2 = df2.set_index('문제고유번호').loc[고유번호필터링_2].reset_index()
    df2 = sort_allen_except_num(df2)
    df2 = df2.reset_index()
    고유번호정렬_2 = df2['문제고유번호'].tolist()
    
    # 두 DataFrame 고유번호만 합치기
    requested_list = df1['문제고유번호'].tolist() + df2['문제고유번호'].tolist()
    df.rename(columns={'correctRate':'정답률'}, inplace=True)
    
    whattime = datetime.now().strftime("%Y%m%d_%H%M%S")
    save = fr'{save_folder}\[{파일명앞에붙을이름}] {str(whattime)[-11:-2]}'

    # 틀린문제 유사한 문제 찾기
    results = find_similar_questions(requested_list, tmp_df_all)

    # docx 파일 생성 후 pdf 파일 생성 (시험)
    question_data = making_word_file(df,requested_list, save+'_시험.docx', whattime, 시험지제목, results)
    convert_doc_to_pdf(save+'_시험.docx', save+'_시험.pdf')


def get_user_input():
    def submit():
        # 두 입력을 받아 리스트로 변환
        고유번호필터링_1 = [int(x.strip()) for x in entry_ids_1.get().split(",") if x.strip()]
        고유번호필터링_2 = [int(x.strip()) for x in entry_ids_2.get().split(",") if x.strip()]
        파일명앞에붙을이름 = entry_prefix.get()
        시험지제목 = entry_title.get()
        root.quit()
        root.destroy()
        main(고유번호필터링_1, 고유번호필터링_2, 파일명앞에붙을이름, 시험지제목, save_folder, file)

    root = tk.Tk()
    root.title("Input Data")

    save_folder = "<저장경로폴더>"
    file = acm_pkl_file_name

    frame = ttk.Frame(root, padding="10")
    frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

    ttk.Label(frame, text="Enter first set of unique IDs (comma-separated):").grid(row=0, column=0, sticky=tk.W)
    entry_ids_1 = ttk.Entry(frame, width=50)
    entry_ids_1.grid(row=0, column=1, pady=5)

    ttk.Label(frame, text="Enter second set of unique IDs (comma-separated):").grid(row=1, column=0, sticky=tk.W)
    entry_ids_2 = ttk.Entry(frame, width=50)
    entry_ids_2.grid(row=1, column=1, pady=5)

    ttk.Label(frame, text="Enter prefix for the file name:").grid(row=2, column=0, sticky=tk.W)
    entry_prefix = ttk.Entry(frame, width=50)
    entry_prefix.grid(row=2, column=1, pady=5)

    ttk.Label(frame, text="Enter the title for the exam paper:").grid(row=3, column=0, sticky=tk.W)
    entry_title = ttk.Entry(frame, width=50)
    entry_title.grid(row=3, column=1, pady=5)

    ttk.Button(frame, text="Submit", command=submit).grid(row=4, column=0, columnspan=2, pady=10)

    root.mainloop()

if __name__ == "__main__":
    get_user_input()



문제고유번호 24020108 쿼리: 병무청은 병역판정검사대상자 `A`의 정확한 신체검사 판정을 위해 `A`의 질병에 대한 확인이 필요하다고 판단하여 오랫동안 `A`를 진료한 의료기관의 장에게 `A`에 대한 진료기록 및 치료 관련 기록의 제출을 요구하였다. 의료기관장의 조치는?
Tokenized query: ['병', '##무', '##청', '##은', '병', '##역', '##판', '##정', '##검', '##사', '##대', '##상', '##자', '[UNK]', 'A', '[UNK]', '의', '정', '##확', '##한', '신', '##체', '##검', '##사', '판', '##정을', '위해', '[UNK]', 'A', '[UNK]', '의', '질', '##병', '##에', '대한', '확인', '##이', '필', '##요', '##하다', '##고', '판', '##단', '##하여', '오', '##랫', '##동안', '[UNK]', 'A', '[UNK]', '를', '진', '##료', '##한', '의', '##료', '##기', '##관', '##의', '장', '##에게', '[UNK]', 'A', '[UNK]', '에', '대한', '진', '##료', '##기', '##록', '및', '치', '##료', '관련', '기', '##록', '##의', '제', '##출', '##을', '요', '##구', '##하였다', '.', '의', '##료', '##기', '##관', '##장의', '조', '##치는', '?']

문제고유번호 24020136 쿼리: 33세 여자가 1개월 전부터 목이 붓고 열이 나서 병원에 왔다. 손이 떨리고 두근거림이 있다고 한다. 1개월간 체중이 2 kg 빠졌다고 한다. 키 162 cm, 체중 46 kg이다. 혈압 110/80 mmHg, 맥박 120회/분, 체온 37.5 ℃이다. 갑상샘이 전체적으로 커져 있고 심한 압통이 있다. 검사 결과는 다음과 같다. 진단은?
Tokenized q

In [336]:
def 모아찍기_4쪽(input_pdf, output_pdf, pages_per_sheet=4):
    # PDF 문서를 엽니다
    doc = fitz.open(input_pdf)

    # 한 시트에 들어갈 페이지 수를 기준으로 결과 페이지의 크기를 설정합니다
    sheet_height = 842 * 2  # 두 페이지의 높이 (세로로 두 페이지)
    sheet_width = 595 * 2   # 두 페이지의 너비 (가로로 두 페이지)
    result_page_rect = fitz.Rect(0, 0, sheet_width, sheet_height)

    # 새 PDF 문서를 생성합니다
    new_doc = fitz.open()

    # 모든 페이지를 필요한 시트 수만큼 처리합니다
    for sheet_index in range(0, len(doc), pages_per_sheet):
        result_page = new_doc.new_page(width=result_page_rect.width, height=result_page_rect.height)

        # 한 시트에 들어갈 페이지를 처리합니다
        for i in range(pages_per_sheet):
            page_index = sheet_index + i
            if page_index >= len(doc):  # 문서의 페이지 수를 초과하는 경우 반복을 종료합니다
                break
            doc.load_page(page_index)
            
            # 각 페이지의 위치를 계산합니다
            row = i // 2  # 페이지가 들어갈 행 위치
            col = i % 2   # 페이지가 들어갈 열 위치
            page_rect = fitz.Rect(col * 595, row * 842, (col + 1) * 595, (row + 1) * 842)
            result_page.show_pdf_page(page_rect, doc, page_index)

    # 결과 PDF를 저장합니다
    new_doc.save(output_pdf)

    new_doc.close()
    doc.close()

def 모아찍기_2쪽(input_pdf, output_pdf, pages_per_sheet=2):
	# PDF 문서를 엽니다
	doc = fitz.open(input_pdf)

	# 한 시트에 들어갈 페이지 수를 기준으로 결과 페이지의 크기를 설정합니다
	sheet_height = 842  # 한 페이지의 높이 (세로로 한 페이지)
	sheet_width = 595 * 2   # 두 페이지의 너비 (가로로 두 페이지)
	result_page_rect = fitz.Rect(0, 0, sheet_width, sheet_height)

	# 새 PDF 문서를 생성합니다
	new_doc = fitz.open()

	# 모든 페이지를 필요한 시트 수만큼 처리합니다
	for sheet_index in range(0, len(doc), pages_per_sheet):
		# 결과 페이지를 생성합니다
		result_page = new_doc.new_page(width=result_page_rect.width, height=result_page_rect.height)

		# 한 시트에 들어갈 페이지를 처리합니다
		for i in range(pages_per_sheet):
			page_index = sheet_index + i
			if page_index >= len(doc):  # 문서의 페이지 수를 초과하는 경우 반복을 종료합니다
				break
			doc.load_page(page_index)
			
			# 각 페이지의 위치를 계산합니다
			col = i % 2  # 페이지가 들어갈 열 위치
			page_rect = fitz.Rect(col * 595, 0, (col + 1) * 595, 842)  # 페이지는 세로로 한 줄에 나열
			result_page.show_pdf_page(page_rect, doc, page_index)  # 수정: show_pdf_page

	# 결과 PDF를 저장합니다
	new_doc.save(output_pdf)

	new_doc.close()
	doc.close()
